#  А/B-тестирование

***Описание проекта***  
***
В нашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.  
**Цель исследования**: провести оценку результатов A/B-теста по внедрению новой платежной системы в работе интернет магазина.  
**Задачи исследования**:
- оценить корректность проведения теста.
    - пересечение тестовой аудитории с конкурирующим тестом;
    - совпадение теста и маркетинговых событий, другие проблемы временных границ теста.
- проанализировать результаты теста.  


***Описание данных***  
***
**Техническое задание**  
Название теста: recommender_system_test;  
Группы: А (контрольная), B (новая платёжная воронка);  
Дата запуска: 2020-12-07;  
Дата остановки набора новых пользователей: 2020-12-21;  
Дата остановки: 2021-01-04;  
Аудитория: 15% новых пользователей из региона EU;  
Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;  
Ожидаемое количество участников теста: 6000.  
Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%:  
- конверсии в просмотр карточек товаров — событие product_page  
- просмотры корзины — product_cart  
- покупки — purchase.  

Загрузите данные теста, проверьте корректность его проведения и проанализируйте полученные результаты.
***
*ab_project_marketing_events.csv* — календарь маркетинговых событий на 2020 год;  
Структура файла:  

    name — название маркетингового события;  
    regions — регионы, в которых будет проводиться рекламная кампания;  
    start_dt — дата начала кампании;  
    finish_dt — дата завершения кампании.

*final_ab_new_users.csv* — все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года;  
Структура файла:

    user_id — идентификатор пользователя;
    first_date — дата регистрации;
    region — регион пользователя;
    device — устройство, с которого происходила регистрация.

*final_ab_events.csv* — все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года;  
Структура файла:

    user_id — идентификатор пользователя;
    event_dt — дата и время события;
    event_name — тип события;
    details — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.
    
*final_ab_participants.csv* — таблица участников тестов.  
Структура файла:

    user_id — идентификатор пользователя;
    ab_test — название теста;
    group — группа пользователя.

In [82]:
#импортируем необходимые для работы с проектом библиотеки
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
import numpy as np
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')
import math as mth
import plotly.express as px
from plotly import graph_objects as go

 ## 1. Загрузим данные, изучим общую информацию и выполним предобработку данных

In [83]:
#сохраним исходные файлы в переменные с которыми в будем работать
ab_project_marketing_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')
display(ab_project_marketing_events.head())
display(ab_project_marketing_events.info())


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


None

In [84]:
#приведем столбцы с датой к корректному типу данных 
ab_project_marketing_events['start_dt'] = pd.to_datetime(ab_project_marketing_events['start_dt'], format ='%Y-%m-%d')
ab_project_marketing_events['finish_dt'] = pd.to_datetime(ab_project_marketing_events['finish_dt'], format ='%Y-%m-%d')

In [85]:
#проверим на наличие пропуска данных и на явные дубликаты
print('Количество пропущенных значений в датасете:', ab_project_marketing_events.isna().mean())
print(' ')
print('Количество явных дубликатов в датасете:', ab_project_marketing_events.duplicated().sum())

Количество пропущенных значений в датасете: name         0.0
regions      0.0
start_dt     0.0
finish_dt    0.0
dtype: float64
 
Количество явных дубликатов в датасете: 0


In [86]:
final_ab_new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')
display(final_ab_new_users.head())
display(final_ab_new_users.info())

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


None

In [87]:
#приведем столбец с датой к корректному типу данных 
final_ab_new_users['first_date'] = pd.to_datetime(final_ab_new_users['first_date'], format ='%Y-%m-%d')

In [88]:
#проверим на наличие пропуска данных и на явные дубликаты
print('Количество пропущенных значений в датасете:', final_ab_new_users.isna().mean())
print(' ')
print('Количество явных дубликатов в датасете:', final_ab_new_users.duplicated().sum())

Количество пропущенных значений в датасете: user_id       0.0
first_date    0.0
region        0.0
device        0.0
dtype: float64
 
Количество явных дубликатов в датасете: 0


In [89]:
final_ab_events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')
display(final_ab_events.head())
display(final_ab_events.info())

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


None

In [90]:
#приведем столбец с датой и временем к корректному типу данных 
final_ab_events['event_dt'] = pd.to_datetime(final_ab_events['event_dt'], format ='%Y-%m-%d %H:%M:%S')

In [91]:
#проверим на наличие пропуска данных и на явные дубликаты
print('Количество пропущенных значений в датасете:', final_ab_events.isna().mean())
print(' ')
print('Количество явных дубликатов в датасете:', final_ab_events.duplicated().sum())

Количество пропущенных значений в датасете: user_id       0.000000
event_dt      0.000000
event_name    0.000000
details       0.857512
dtype: float64
 
Количество явных дубликатов в датасете: 0


In [92]:
final_ab_participants = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')
display(final_ab_participants.head())
display(final_ab_participants.info())

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


None

In [93]:
#проверим на наличие пропуска данных и на явные дубликаты
print('Количество пропущенных значений в датасете:', final_ab_participants.isna().mean())
print(' ')
print('Количество явных дубликатов в датасете:', final_ab_participants.duplicated().sum())

Количество пропущенных значений в датасете: user_id    0.0
group      0.0
ab_test    0.0
dtype: float64
 
Количество явных дубликатов в датасете: 0


Данные всех датафреймов обработанны и готовы к дальнешей работе:  
- произведена замена типов данных;  
- проверка на явные дубликаты;  
- осуществлена проверка наличия пропусков данных, выявлен большой объем (85,75%) пропущенных значений в столбце details датасета final_ab_events Т.к. в данной колонке указана дополнительная информация, наличие пропусков может быть обусловлено отсутствием необходимости указания данной инфо, поэтому оставим пропуски как есть.  

## 2. Оценим корректность проведения теста. 
Проверим соответствие данных требованиям технического задания.

In [94]:
final_ab_participants['ab_test'].unique()

array(['recommender_system_test', 'interface_eu_test'], dtype=object)

Нам предоставлены данные для двух тестов: interface_eu_test и recommender_system_test.  
По тех.заданию нам потребуется только recommender_system_test

In [95]:
#проверим есть ли группы для необходимого нам теста
final_ab_participants[['ab_test', 'group']].groupby('ab_test').agg(set)

,group
ab_test,
interface_eu_test,"{A, B}"
recommender_system_test,"{A, B}"


Нам представлены группы: А (контрольная), B (новая платёжная воронка) соответсвует тех.заданию.  

In [96]:
print('количество уникальных пользователей', final_ab_participants['user_id'].nunique())


количество уникальных пользователей 16666


In [97]:
#количество уникальных пользователей по группам
final_ab_participants.groupby(['ab_test', 'group']).agg({'user_id': 'nunique'})


user_id
ab_test                 group         
interface_eu_test       A         5831
                        B         5736
recommender_system_test A         3824
                        B         2877

In [98]:
#Считаем количество пользователей, которые находятся в обоих тестах
users_in_group = final_ab_participants.groupby('user_id').agg({'ab_test': 'nunique'})\
                                      .sort_values(by = 'ab_test', ascending = False).reset_index()
users_in_group.sample(5)

,user_id,ab_test
9724,2818C4BDDACA855C,1
10864,096C866A2E46D20D,1
14217,794D683C2E501D1E,1
14202,790CC1D345829B2E,1
9173,2EF6C327513740D6,1


In [99]:
#Выделяем пользователей, которые попали в два теста сразу
users_in_2_group = users_in_group.loc[users_in_group['ab_test'] == 2]
print('количество пользователей, попавших в два теста', users_in_2_group['user_id'].nunique())
users_in_2_group = users_in_2_group['user_id']

количество пользователей, попавших в два теста 1602


In [100]:
#Выделим из общего датасета пользователей, которые находятся в обоих тестах
users_in_2_group = final_ab_participants.query('user_id in @users_in_2_group').reset_index(drop=True)
#Колчество этих пользователей по тестам и группам
users_in_2_group.groupby(['ab_test', 'group']).agg({'user_id': 'nunique'})

user_id
ab_test                 group         
interface_eu_test       A          819
                        B          783
recommender_system_test A          921
                        B          681

In [101]:
#оставим для дальнейшей работы только пользователей участвующих в  тесте recommender_system_test
final_ab_participants = final_ab_participants.query('ab_test == "recommender_system_test"')


In [102]:
#проверим пересекаются ли пользователи в контрольных и экспериментальной группах
control1 = final_ab_participants.query('group == "A"')
exp = final_ab_participants.query('group == "B"')
display(set(exp['user_id']).intersection(set(control1['user_id'])))
display(final_ab_participants.groupby('user_id').agg({'group' : 'nunique'}).query('group > 1'))

set()

,group
user_id,


In [103]:
#Объединим таблицы

final_ab_users = final_ab_events.merge(final_ab_new_users, on = 'user_id')

final_ab_users.head()

,user_id,event_dt,event_name,details,first_date,region,device
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99,2020-12-07,N.America,iPhone
1,E1BDDCE0DAFA2679,2020-12-09 06:21:35,purchase,9.99,2020-12-07,N.America,iPhone
2,E1BDDCE0DAFA2679,2020-12-25 08:26:03,purchase,499.99,2020-12-07,N.America,iPhone
3,E1BDDCE0DAFA2679,2020-12-07 20:22:03,login,NaN,2020-12-07,N.America,iPhone
4,E1BDDCE0DAFA2679,2020-12-09 06:21:35,login,NaN,2020-12-07,N.America,iPhone


In [104]:
final_users_recommender_system_test = final_ab_users.query('user_id in @final_ab_participants.user_id')\
                                                    .merge(final_ab_participants, on = 'user_id')
final_users_recommender_system_test.head()

,user_id,event_dt,event_name,details,first_date,region,device,group,ab_test
0,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99,2020-12-07,EU,Android,A,recommender_system_test
1,831887FE7F2D6CBA,2020-12-09 02:19:17,purchase,99.99,2020-12-07,EU,Android,A,recommender_system_test
2,831887FE7F2D6CBA,2020-12-07 06:50:30,product_cart,NaN,2020-12-07,EU,Android,A,recommender_system_test
3,831887FE7F2D6CBA,2020-12-08 10:52:27,product_cart,NaN,2020-12-07,EU,Android,A,recommender_system_test
4,831887FE7F2D6CBA,2020-12-09 02:19:17,product_cart,NaN,2020-12-07,EU,Android,A,recommender_system_test


In [105]:
#Проверим дату запуска теста
print('Дата первого события: {}'.format(final_users_recommender_system_test['event_dt'].min()))

Дата первого события: 2020-12-07 00:05:57


Тех.заданием предусмотрена дата запуска теста соответсвующая дате начала событий

In [106]:
#Проверим дату остановки набора новых пользователей.
print('Конечная дата регистрации пользователей: {}'.format(final_users_recommender_system_test['first_date'].max()))

Конечная дата регистрации пользователей: 2020-12-21 00:00:00


Конечная дата регистрации пользователей соответсвует тех.заданию.

In [107]:
#Проверим дату остановки теста
print('Дата последнего события: {}'.format(final_users_recommender_system_test['event_dt'].max()))

Дата последнего события: 2020-12-30 12:42:57


Тех.заданием предусмотрена дата окончания теста 2021-01-04, дата последнего события меньше даты остановки теста на 5 дней

In [108]:
print('Процент новых пользователей из региона EU: {}'.format(
    round(final_ab_new_users.query('user_id in @final_ab_participants.user_id and region == "EU"').shape[0]\
          /final_ab_new_users.query('region == "EU" and first_date >= @final_users_recommender_system_test.first_date.min()\
          and first_date <= @final_users_recommender_system_test.first_date.max()').shape[0]*100, 2))
     )

Процент новых пользователей из региона EU: 15.0


Процент новых пользователей из региона EU соответсвует тех.заданию.

In [109]:
print('Количество участников теста: {}'.format(final_users_recommender_system_test['user_id'].nunique()))

Количество участников теста: 3675


Фактическое количество участников теста существенно ниже ожидаемого, это может повлиять на чистоту теста

In [110]:
# создадим новый столбец, в котором запишем лайфтайм каждого события
final_users_recommender_system_test['lifetime'] = (final_users_recommender_system_test['event_dt']\
                                                   - final_users_recommender_system_test['first_date']).dt.days 
display(final_users_recommender_system_test.sample(5))

,user_id,event_dt,event_name,details,first_date,region,device,group,ab_test,lifetime
6784,124B6B7F19141CCC,2020-12-20 19:12:27,product_page,NaN,2020-12-18,EU,Android,A,recommender_system_test,2
8507,DCAA50AF3E368063,2020-12-21 07:20:13,purchase,4.99,2020-12-20,EU,iPhone,A,recommender_system_test,1
6103,EB0712682DC1C12B,2020-12-17 21:43:07,login,NaN,2020-12-17,EU,Mac,A,recommender_system_test,0
19311,89EF9F0C1676188A,2020-12-20 09:46:54,product_page,NaN,2020-12-16,EU,Android,B,recommender_system_test,4
914,89545C7F903DBA34,2020-12-08 15:39:01,purchase,4.99,2020-12-07,EU,Android,B,recommender_system_test,1


In [111]:
#отсеим события, которые выходят за рамки рассматриваемого периода
final_users_recommender_system_test = final_users_recommender_system_test.query('lifetime < 15')

In [112]:
#проверка активностей во время теста

ab_project_marketing_events.query('start_dt > "2020-12-07" and finish_dt < "2021-01-04"')

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03


Одно запланированное событие попадает в диапазон наших дат: Christmas&New Year Promo, соответственно это может оказать негативное влияние на результат эксперимента, т.к. оно может отвлекать от целевых событий нашего эксперимента, а значит может ухудшить показатели конверсии.

In [113]:
#Удостоверимся, что нет пользователей, участвующих в двух группах теста одновременно.
A = final_users_recommender_system_test.query('group=="A"')['user_id'].unique()
B = final_users_recommender_system_test.query('group=="B"')['user_id'].unique()
C = np.intersect1d(A, B)
print('Количество пересекающихся пользователей в группе А и В', len(C))
print('Количество пользователей в группе А', len(A))
print('Количество пользователей в группе В', len(B))

Количество пересекающихся пользователей в группе А и В 0
Количество пользователей в группе А 2747
Количество пользователей в группе В 928


Мы видим, что кол-во пользователей группы А значительно превышает кол-во пользователей группы В (практически в 3 раза). Это говорит нам о неравномерности разбиения данных по группам. 

Вывод:  
- В данных оставлены пользователи нужного теста и заявленные группы A и B, предусмотренные с тех.заданием, пользователи в группах не пересекаются.   
- Тех.заданием предусмотрена дата запуска теста соответсвующая дате начала событий  
- Конечная дата регистрации пользователей соответсвует тех.заданию.  
- Тех.заданием предусмотрена дата окончания теста 2021-01-04, дата последнего события меньше даты остановки теста на 5 дней
- Процент новых пользователей из региона EU соответсвует тех. заданиюм.  
- Ожидаемое кол-во участников теста существенно ниже ожидаемого, это может повлиять на чистоту теста.  
- Мы проверили, есть ли в данных события старше 14 дней и очистили датасет от таких событий.  

Одно запланированное событие попадает в диапазон наших дат: Christmas&New Year Promo, соответственно это может оказать негативное влияние на результат эксперимента.  

Количество пользователей группы А значительно превышает кол-во пользователей группы В (практически в 3 раза). Это говорит нам о неравномерности разбиения данных по группам.

In [115]:
!pip install bokeh
!pip install folium
!pip install psycopg2
!pip install plotly

In [122]:
%matplotlib inline
import matplotlib.pyplot as plt

## 3. Проведем исследовательский анализ данных.

### 3.1 Количество событий на пользователя одинаково распределены в выборках?

In [123]:
# строим гистограмму распределение кол-ва действий
fig = px.histogram(final_users_recommender_system_test, # загружаем данные
                   x='event_name', # указываем столбец с данными для оси X
                   color='group', # обозначаем категорию для разделения цветом
                   title='Распределение количества действий по группам',) # указываем заголовок
fig.update_xaxes(title_text='Событие') # подпись для оси X
fig.update_yaxes(title_text='Количество событий') # подпись для оси Y
fig.show() # выводим график

In [119]:
event_mean_A = final_users_recommender_system_test.query('group == "A"').pivot_table(index=['user_id', 'event_name'],\
               values='event_dt', aggfunc='count').reset_index().groupby('event_name').agg({'event_dt':'mean'}).reset_index()\
               .round(2)
event_mean_B = final_users_recommender_system_test.query('group == "B"').pivot_table(index=['user_id', 'event_name'],\
               values='event_dt', aggfunc='count').reset_index().groupby('event_name').agg({'event_dt':'mean'}).reset_index()\
               .round(2)
fig = go.Figure(data=[go.Bar(name='Group A', x=event_mean_A['event_name'].to_list(),
                             y=event_mean_A['event_dt'].to_list(),
                             text=event_mean_A['event_dt'].to_list(),
                             textposition='auto'),
                      go.Bar(name='Group B', x=event_mean_B['event_name'].to_list(),
                             y=event_mean_B['event_dt'].to_list(),
                             text=event_mean_B['event_dt'].to_list(),
                             textposition='auto')
])

fig.update_layout(title='График среднего количества взаимодействий с карточкой на одного пользователя')
fig.update_xaxes(title_text='Событие') # подпись для оси X
fig.update_yaxes(title_text='Количество событий') # подпись для оси Y
fig.show()

Пользователи в группе А событий совершают больше, чем в группе В. Также стоит отметить, что распределения очень схожи между собой.

### 3.2 Как число событий в выборках распределено по дням?

In [116]:
# строим гистограмму распределение кол-ва действий
fig = px.histogram(final_users_recommender_system_test, # загружаем данные
                   x='event_dt', # указываем столбец с данными для оси X
                   color='group', # обозначаем категорию для разделения цветом
                   title='Распределение количества событий по датам',) # указываем заголовок
fig.update_xaxes(title_text='Дата') # подпись для оси X
fig.update_yaxes(title_text='Количество событий') # подпись для оси Y
fig.show() # выводим график

In [37]:
# строим гистограмму распределение кол-ва действий
fig = px.histogram(final_users_recommender_system_test.groupby(['first_date', 'group'])['user_id'].nunique().reset_index(), # загружаем данные
                   y='user_id',
                   x='first_date', # указываем столбец с данными для оси X
                   color='group', # обозначаем категорию для разделения цветом
                   title='Распределение количества пользователей в группах по дате регистрации',) # указываем заголовок
fig.update_xaxes(title_text='Дата регистрации') # подпись для оси X
fig.update_yaxes(title_text='Количество пользователей') # подпись для оси Y
fig.show() # выводим график

События в выборках распределены по дням отличным друг от друга образом. В группе А пользователи были активны во второй половине исследуемого промежутка времени, что связано со скачком регистрации новых пользователей с 14-12-2020, а для пользователей группы В распределение по времени на графике выглядит более равномерно.  
Необходимо отметить большое количество зарегистрированных.   
Также стоит учитывать и тот факт, что конец декабря - это время активности аудитории по причине приближающихся праздников, когда увеличевается спрос на «подарочные» товары. Данные события могут повлияться на результаты тестирования.  
Значения количества действий для группы А больше, чем у группы В, что подтверждает факт несхожего распределения количества событий на пользователя в группах. 

### 3.3 Как меняется конверсия в воронке в выборках на разных этапах?

In [38]:
#применим группировку событий и подсчет количества уникальных пользователей
user_count_a = final_users_recommender_system_test.query('group == "A"').groupby(['event_name'])\
                                                  .agg({'user_id':'nunique'})\
                                                  .sort_values(by='user_id', ascending=False).reset_index()

user_count_a

,event_name,user_id
0,login,2747
1,product_page,1780
2,purchase,872
3,product_cart,824


воронка должна заканчиваться целевым действием (оплата), но на графике - последнее событие продуктовая карта, т.е. основная цель пользователя - просмотр продуктовой корзины, что является не совсем корректно. Тут стоит поменять местами

In [39]:
#поменяем местами порядок событий и добам столбцы с долей пользователей и конверсией
user_count_a = user_count_a.reindex([0, 1, 3, 2])
user_count_a['ratio'] = round(user_count_a['user_id'] / final_users_recommender_system_test.query('group == "A"')['user_id']\
                                                                                           .nunique(), 2)
user_count_a['shift'] = user_count_a['user_id'].shift(periods=1, axis=0, fill_value=user_count_a.loc[0,'user_id'])
user_count_a['conv'] = round((user_count_a['user_id'] / user_count_a['shift']),3)
user_count_a.drop('shift', axis='columns', inplace=True)
user_count_a

,event_name,user_id,ratio,conv
0,login,2747,1.00,1.000
1,product_page,1780,0.65,0.648
3,product_cart,824,0.30,0.463
2,purchase,872,0.32,1.058


In [40]:
user_count_b = final_users_recommender_system_test.query('group == "B"').groupby(['event_name'])\
                                                  .agg({'user_id':'nunique'})\
                                                  .sort_values(by='user_id', ascending=False).reset_index()
user_count_b['ratio'] = round(user_count_b['user_id'] / final_users_recommender_system_test.query('group == "B"')['user_id']\
                                                                                           .nunique(), 2)
user_count_b['shift'] = user_count_b['user_id'].shift(periods=1, axis=0, fill_value=user_count_b.loc[0,'user_id'])
user_count_b['conv'] = round((user_count_b['user_id'] / user_count_b['shift']),3)
user_count_b.drop('shift', axis='columns', inplace=True)
user_count_b

,event_name,user_id,ratio,conv
0,login,927,1.00,1.000
1,product_page,523,0.56,0.564
2,purchase,256,0.28,0.489
3,product_cart,255,0.27,0.996


In [43]:
# построим воронку событий для каждой из групп
fig = go.Figure(go.Funnel(
    name='Группа A',
    y = user_count_a['event_name'],
    x = user_count_a['user_id'],
    textinfo = 'value+percent initial'))

fig.add_trace(go.Funnel(
    name='Группа B',
    y = user_count_b['event_name'],
    x = user_count_b['user_id'],
    textinfo = 'value+percent initial'))
#fig.update_xaxes(title_text='Дата') # подпись для оси X
fig.update_yaxes(title_text='Название событий') # подпись для оси Y
fig.update_layout(title={'text' : 'Воронка группы А — контрольная, B — новая платёжная воронка', 
                         'x':0.50,
                         'y':0.88,
                         'xanchor': 'center'
                        })
fig.show()

Вывод:  
Процент пользователей и конверсия между группами отличаются незначительно, но на всех этапах изменяется в сторону уменьшения, таким образом новый способ оплаты не привел к увеличению конверсии каждой метрики. 

### 3.4 Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

1. Фактическое кол-во участников теста существенно ниже ожидаемого.
2. Группы А и В крайне неравномерны распределены.
3. Время проведения теста пересекается с маркетинговым событием Christmas&New Year Promo.
4. Отсутсвие данных с 30 декабря 

## 4 Проведем оценку результатов A/B-тестирования

### 4.1 Что можно сказать про результаты A/B-тестирования?

На воронке стоит отметить следующее:
- по событию product_page в группе А конверсия составила 65%, в группе В она снизилась до 56%.
- по событию purchase ситуация конверсия снизилась с 32% до 28%.
- по событию product_cart конверсия снизилась с 30% до 27,5%.  

Согласно тех.заданию конверсия должна была увеличиться на 10% по каждому событию, но мы видим что она упала по всем пунктам. Это может говорить о несостоятельности новой платежной воронки и провале теста.

### 4.2 Проверим статистическую разницу долей z-критерием

Для проверки гипотезы о равенстве пропорций двух генеральных совокупностей по их выборкам примем уровень значимости **alpha = 0.05** и сформулируем:  
    - нулевую гипотезу H0: между долями нет значимой разницы   
    - альтернативную гипотезу H1: : между долями есть значимая разница. 



In [41]:
final = final_users_recommender_system_test.drop_duplicates('user_id')
users_count_by_group = final.query('event_name == "purchase" or event_name == "product_cart" or event_name == "product_page" \
                                    or event_name == "login"').groupby('group')['user_id'].nunique()
users_by_events = final_users_recommender_system_test.pivot_table(index='group', columns='event_name', values='user_id', 
                                                                  aggfunc='nunique').reset_index()
users_by_events['user_count'] = users_by_events['group'].apply(lambda x: users_count_by_group.loc[x])
users_by_events = users_by_events.set_index('group')
users_by_events = users_by_events[['login', 'product_page', 'product_cart', 'purchase', 'user_count']]
users_by_events

event_name,login,product_page,product_cart,purchase,user_count
group,,,,,
A,2747,1780,824,872,2747
B,927,523,255,256,928


In [42]:
# функция для определения статистической разницы долей z-критерием
alpha = 0.05
def st_test(part0, part1, total0, total1):  
    
    p1 = part0 / total0 
    p2 = part1 / total1 
    print(part0, part1,total0 , total1)
    p_combined = (part0 + part1) / (total0 + total1) 
    difference = p1 - p2 
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/total0 + 1/total1))
    distr = st.norm(0, 1) 
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    return p_value

In [43]:
AB_pivot = pd.DataFrame(columns=['Группа_1', 'Группа_2', 'Этап', 'p_alpha', 'p_value', 'Результат'])
bonferroni_alpha = alpha / 4
result_options = ["Отвергаем нулевую гипотезу - между долями есть статзначимая разница",
                  "Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными"]

group_1 = "A"
current_row = users_by_events.loc[group_1]
group_2 = "B"
for event_number in range(4):
    curr_event = users_by_events.columns[event_number]
    p_value_result = st_test(current_row[curr_event], 
                                   users_by_events[curr_event][group_2],
                                   current_row['user_count'],
                                   users_by_events['user_count'][group_2]
                           )
    new_row = {'Группа_1': group_1,
               'Группа_2': group_2,
               'Этап': users_by_events.columns[event_number],
               'p_alpha': bonferroni_alpha,
               'p_value': round(p_value_result, 6),
               'Результат': result_options[p_value_result >= bonferroni_alpha]}
    AB_pivot = AB_pivot.append([new_row])

AB_pivot.index = range(len(AB_pivot))
AB_pivot

2747 927 2747 928
1780 523 2747 928
824 255 2747 928
872 256 2747 928


,Группа_1,Группа_2,Этап,p_alpha,p_value,Результат
0,A,B,login,0.0125,0.085299,"Не получилось отвергнуть нулевую гипотезу, нет..."
1,A,B,product_page,0.0125,0.000004,Отвергаем нулевую гипотезу - между долями есть...
2,A,B,product_cart,0.0125,0.145348,"Не получилось отвергнуть нулевую гипотезу, нет..."
3,A,B,purchase,0.0125,0.017592,"Не получилось отвергнуть нулевую гипотезу, нет..."


При помощи z_test мы проверили, есть ли между долями групп существенная разница и выяснили:
- по событиям login, product_cart и purchase разница между долями уникальных посетителей, побывавших на этапе воронки, не является статистически значимой. 
- по событию product_page статистическая разница в долях уникальных посетителей, побывавших на этапе воронки, существенная.

## 5 Общий вывод

Для определения какая рекомендательная система лучше, был проведен A/B-эксперимент, где А – контрольная группа, В – группа с новой платёжной воронкой, после которого необходимо было сопоставить доли пользователей, проходящих на следующий шаг воронки, для каждой группы.  
События воронки: login, product_page, product_cart, purchase. Конверсия в каждый этап воронки для группы B меньше, чем у группы А. Из этого можно сделать вывод, что новая платежная система не увеличивает, а ухудшает конверсию пользователей в целевое действие. 

После проверки гипотез о равенстве долей, можно сделать вывод, что доли пользователей, проходящих на этап product_page и различны, где конверсия группы В ниже, чем А. Доли пользователей, проходящих на этап login, product_cart и purchase одинаковы для группы А и В.

Аспекты, которые оказали влияние на результаты исследования:

- Пользователи распределились между сегментами неравномерно;
- Количество пользователей, попавших в исследование, не соответствует ТЗ.
- Нет данных после 30 декабря, хотя тест проводился вплоть до 4 января.
- время проведения теста совпадало с маркетинговыми активностями. 

Для получения более точных результатов и принятия решений, по использованию нового функционала необходимо провести повторное тестирование, с учетом вышеупомянутых аспектов.